In [1]:
import numpy as np
from math import pi, cos
from numpy import linalg as LA
%run Functions.ipynb

## 缺分解 diagonalize graph laplacian

In [6]:
def Update_b(b_old, l_old, a_old, v_old, y_ave, n, K):
    lr_b = 1e-9
    db = (n*b_old + n*(K/v_old)*y_ave)/(l_old + K/v_old) - n*(b_old/l_old)
    b = b_old + (lr_b/n)*db
    
    return b

def Update_l(b_old, l_old, a_old, v_old, y_avg, n, K, phi, m_old):
    lr_l = 1e-13
    tmp_1 = np.sum(m_old**2)
    tmp_2 = sum([1/(l_old + K/v_old + a_old*phi[i]) for i in range(n)])
    tmp_3 = sum([1/(l_old + a_old*phi[i]) for i in range(n)])
    dl = (b_old**2/l_old**2 + tmp_3 - tmp_1 - tmp_2)/2
    l = l_old + (lr_l/n)*dl
    return l

def Update_a(l_old, a_old, v_old, n, K, phi, m_old):
    lr_a = 1e-9
    tmp_1 = []
    for i in range(int(np.sqrt(n))):
        for j in range(int(np.sqrt(n))):
            tmp_1.append(m_old[i][(j+1)%3] + m_old[i][(j-1)%3] + m_old[(i+1)%3][j] + m_old[(i-1)%3][j])

    tmp_1 = sum(tmp_1)    
    tmp_2 = sum([phi[i]/(l_old + K/v_old + a_old*phi[i]) for i in range(n)])
    tmp_3 = sum([phi[i]/(l_old + a_old*phi[i]) for i in range(n)])
    
    da = (tmp_3 - tmp_1 - tmp_2)/2
    a = a_old + (lr_a/n)*da
    
    return a

def Update_v(l_old, a_old, v_old, n, K, phi, m_old, degraded_images):
    m_y_diff = sum([np.sum((m_old - y)**2) for y in degraded_images])
    tmp = sum([1/(l_old + K/v_old + a_old*phi[i]) for i in range(n)])
    v = m_y_diff/(n*K) + tmp/n

    return v

In [7]:
def LT_EM(degraded_images):
    ## 獲得影像 size 資訊
    K, h, w = degraded_images.shape ## 這邊影像是方形
    
    n = h*w
    
    ## 初始化參數
    b_old = 0
    l_old = 1e-7
    a_old = 1e-4
    v_old = 1000
    
    b = b_old
    l = l_old
    a = a_old
    v = v_old
    
    phi = [4 - 2*(cos(i*pi/h) + cos(j*pi/h)) for i in range(h) for j in range(w)]
    
    ## 初始化 m
    y_avg = np.mean(degraded_images, axis=0)
    y_ave = np.mean(y_avg)
    m_old = y_avg.copy()
    m = m_old.copy()
    
    ## 中止條件（可能無法）
    while(True):
        ## 更新 m
        for i in range(h):
            for j in range(w):
                neighbor_sum = m_old[i][(j+1)%3] + m_old[i][(j-1)%3] + m_old[(i+1)%3][j] + m_old[(i-1)%3][j]
                m[i][j] = (1/(l_old + a_old*4 + K/v_old)) * (b_old + (K/v_old)*y_avg[i][j] + a_old*neighbor_sum)
    
                
        ## 更新 Variance
        v = Update_v(l_old, a_old, v_old, n, K, phi, m_old, degraded_images)
        
        if(max(abs(np.asarray([b_old-b, l_old-l, a_old-a, v_old-v]))) < 1e-5):
            break
        ## 更新 b, l, a
        b = Update_b(b_old, l_old, a_old, v_old, y_ave, n, K)
        l = Update_l(b_old, l_old, a_old, v_old, y_avg, n, K, phi, m_old)
        a = Update_a(l_old, a_old, v_old, n, K, phi, m_old)
    
        b_old = b
        l_old = l
        a_old = a
        v_old = v
        m_old = m.copy()
    
    return b, l, a, v, m

In [8]:
def Get_ROI_Loc(cy,cx,cr):
    ## Loc 從左上到右下，y放前面（符合 image 讀取）
    x, y, r = int(round(cx)), int(round(cy)), int(round(cr))
    roi_loc = [[x-r-10, y-r-10], [x+r+10, y+r+10] ]
    return np.asarray(roi_loc)

def Get_ROI(image):
    ## 拿一張影像先算 Center 拿來取 ROI
    _, _, _, edge = image_processing(image)
    edge_pts = get_edge_points(edge)
    cx,cy,cr,_ = cf.least_squares_circle((edge_pts))
    
    roi_loc = Get_ROI_Loc(cy,cx,cr)
    return roi_loc

In [11]:
import os
import cv2
import circle_fit as cf


os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/New/Case_4")

files = os.listdir()
files.sort()
files = np.asarray(files)
batches = np.split(files, 297)

predictions = []
for batch in batches:
    images = [cv2.imread(img,0) for img in batch]
    roi_loc = Get_ROI(images[0])
    
    lx, ly = roi_loc[0,:]
    rx, ry = roi_loc[1,:]
    images = np.asarray([image[ly:ry, lx:rx] for image in images])
    
    mean = np.mean(images, axis=0).astype('uint8')
    em   = np.around(LT_EM(images)[4]).astype('uint8')
    
#     cv2.imshow("Mean", mean)
#     cv2.imshow("EM", em)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    _, _, _, edge = image_processing(em)
    points = get_edge_points(edge)
    cx, cy, r, _ = cf.least_squares_circle((points))
    
    print("{}, {}".format(cx+lx, cy+ly))
    predictions.append([cx+lx, cy+ly])

86.32752627960329, 72.68856650709485
86.46849051671802, 72.7023160335279
86.44021285134417, 72.70504613844892
86.34779771339613, 72.63575618705099
86.40629936176506, 72.72682905118934
86.47758378773838, 72.71007692129805
86.35637759970098, 72.73256181305356
86.29193186830365, 72.81029888115006
86.41736743552164, 72.90926449239782
86.41218822183095, 72.8094775332049
86.37100625694369, 72.84381319568155
86.34226941576134, 72.74630017970051
86.32941516476083, 72.8375134783947
86.29399984390349, 72.88492782012256
86.37204493195155, 72.7247406329634
86.2689661758555, 72.73991228817846
86.36928169801631, 72.72057859521962
86.39937605170562, 72.77373646948158
86.37930252602328, 72.8521896999783
86.4328508851707, 72.74887948529437
86.40810860260918, 72.7937951765542
86.37313476649292, 72.6974272107276
86.3872159980042, 72.75881082319152
86.4896470545323, 72.73757936597983
86.3436484898793, 72.8118530978058
86.36955008621732, 72.66714512127109
86.40052422308523, 72.69569291655024
86.34586956100

86.29457436928192, 72.63195520447002
86.35480030098631, 72.74419616367086
86.26255460306527, 72.67821587697358
86.3203123919655, 72.72465431828971
86.38816542407116, 72.67786585720654
86.36892609276322, 72.72224142236621
86.29054606522703, 72.70329161802348
86.3529616760799, 72.58692780879952
86.29946361935589, 72.72265900552651
86.38149358797769, 72.70240768392664
86.41553083908394, 72.87590062474379
86.3549916518216, 72.7691457857273
86.34097639578916, 72.79167063995327
86.4248456072234, 72.66753265196027
86.3504282003485, 72.73753867444755
86.36890479542132, 72.81451856500684
86.43425914108357, 72.7499148976448
86.44886348915696, 72.7099453583264
86.3826730733678, 72.68375277611553
86.40061322391482, 72.67379654708125
86.31191981806913, 72.81690578658527
86.49362703783733, 72.71058624725038
86.35770107436855, 72.69178002372388
86.30349272027453, 72.788373569326
86.33198663528157, 72.76250287671397
86.40960239954862, 72.71275609830822
86.3406198201545, 72.60261707752932
86.3782472512

In [12]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (86.37628, 72.80218)
Variance     : (0.00205, 0.00571)
Max Length   : 0.24058, 0.41282 (pixels)
